In [1]:
# import  
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# read training data to a DataFrame df
df = pd.read_csv('train_titanic.csv')

# Preprocessing

# drop misssing 'Embarked' values
df = df.dropna(subset=['Embarked'])

# fill in missing 'Age values'
median_value = df['Age'].median()
df['Age'] = df['Age'].fillna(median_value)

# convert 'Sex' column type to category
df['Sex'] = df['Sex'].astype('category')

# Dummy variable encoding
dummies = pd.get_dummies(df[['Sex']], prefix_sep = '_')

# add new binary 'Female' column to df for numerical analysis
#df = pd.concat([df, dummies], axis = 1)
df['Female'] = dummies['Sex_female']

# convert 'Embarked' column type to category
df['Embarked'] = df['Embarked'].astype('category')

# Dummy variable encoding
dummies = pd.get_dummies(df[['Embarked']], prefix_sep = '_')

# add new binary 'Embarked' columns to df for numerical analysis
df['Embarked_C'] = dummies['Embarked_C']
df['Embarked_S'] = dummies['Embarked_S']

# Fitting a KNN model

X = df.drop(['Survived', 'Name', 'Ticket', 'Cabin', 'Embarked', 'Sex'], axis = 1).values
y = df['Survived'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, \
                                                    random_state = 1, \
                                                    stratify = y)

param_grid = {'n_neighbors': np.arange(1, 50)}
knn = KNeighborsClassifier(n_neighbors = 6)
knn_cv = GridSearchCV(knn, param_grid, cv = 5)
knn_cv.fit(X_train, y_train)

y_pred = knn_cv.predict(X_test)


print(knn_cv.best_params_)
print(knn_cv.best_score_)

print(knn_cv.score(X_test, y_test))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


{'n_neighbors': 17}
0.6672025723472669
0.6404494382022472
[[144  21]
 [ 75  27]]
              precision    recall  f1-score   support

           0       0.66      0.87      0.75       165
           1       0.56      0.26      0.36       102

    accuracy                           0.64       267
   macro avg       0.61      0.57      0.56       267
weighted avg       0.62      0.64      0.60       267

